# 데이터 결합 및 부분 선택

### 주요 내용

1. 데이터 결합
2. index, columns을 활용한 부분 선택 
3. 조건을 활용한 관측치 선택

<br>

### 목표 
1. 복수의 데이터를 적절한 방법으로 결합할 수 있다.
2. 변수 이름 등을 활용하여 부분 데이터를 선택한다.
3. 주제에 맞게 조건을 활용하여 부분 관측치를 선택한다. 


<br>
<hr>
<br>

## 1. DataFrame 형식의 활용

pandas는 데이터를 저장하는 형식 **DataFrame**을 중심으로 구성되어 있음  
pandas의 다양한 함수를 활용해서 데이터를 불러오거나 저장할 수 있고, 분석 과정에서 필요한 전처리나 집계 작업도 가능


In [ ]:
# 라이브러리 불러오기
import pandas as pd

<br>

DataFrame에서 각각의 열, 변수가 하나의 Series로 저장되어 있음  
Series의 메서드와 DataFrame의 메서드 구분 필요  

만약 직접 DataFrame을 만들어야 할 때는 *DataFrame( )* 과 딕셔너리를 활용

In [ ]:
# 딕셔너리를 활용한 DataFrame 생성
df_own = pd.DataFrame(
    
    {'FIRST' : ['A', 'B', 'C', 'D'],
     'SECOND': [7,6,5,8], 
     'THIRD' : pd.date_range('2022-12-05', periods=4, freq='W-SUN')}

) # freq='W-MON' : 매주 월요일
df_own

<br>
<hr>
<br>

## 2. 데이터 결합

### 2.1. concat( )을 활용한 동일 구조 데이터 행 결합

구조는 똑같고 기간이나 상품만 다른 여러 데이터가 있으면 pandas의 *concat()* 으로 결합해서 활용  
함수 안에서 `axis=0`옵션을 활용해서 행 결합(아래로 이어 붙이기)을 할 수 있고, `axis=1`로 열 결합도 가능  
`axis=0`이 기본값며 생략 가능

In [ ]:
# 행 결합
    ## 출처 : 국토교통부 실거래가(http://rtdown.molit.go.kr/)
df_apt1 = pd.read_csv('data/아파트(매매)__실거래가_20210902153616.csv', skiprows=15, encoding='CP949')
df_apt1

In [ ]:
df_apt2 = pd.read_csv('data/아파트(매매)__실거래가_20210902153636.csv', skiprows=15, encoding='CP949')
df_apt2

In [ ]:
df_apt3 = pd.read_csv('data/아파트(매매)__실거래가_20210902153655.csv', skiprows=15, encoding='CP949')
df_apt3

In [ ]:
df_apt = pd.concat([df_apt1, df_apt2, df_apt3], join="inner")
df_apt

<br>

> **DataFrame**에서 행 번호에 해당하는 **index**는 중요한 역할을 합니다.  
예를 들어 아래처럼 index를 확인할 수 있고, 특정 index를 지정해서 관측치를 선택하는 것도 가능합니다. 

In [ ]:
df_apt.index

In [ ]:
# index 0 관측치 선택
df_apt.loc[0]

결합 이전 기존 Index 활용으로 **0** 인덱스 관측치의 중복 발생  
행 결합이나 정렬 이후 인덱스를 재지정하거나 초기화 필요 

In [ ]:
# reset_index()을 활용한 index 초기화
    ## drop=True: 기존 인덱스를 변수로 추가할 지 버릴지 선택
df_apt = df_apt.reset_index(drop=True)
df_apt

In [ ]:
# index 0 관측치 재선택
df_apt.loc[0]

#### [실습]  데이터 결합 및 인덱스 초기화

출처 : [서울시 지하철 호선별 역별 승하차 인원수](http://data.seoul.go.kr/dataList/OA-12914/S/1/datasetView.do)

1. `data`폴더의 `CARD_SUBWAY_MONTH_`로 시작하는 3개 데이터 확인하기  
    


2. 1.의 데이터를 각각 불러와서 저장하고, pd.concat()으로 행 결합하기(encoding='CP949' 활용)


3. index 초기화 하기



In [ ]:
df_card1 = pd.read_csv("./data/CARD_SUBWAY_MONTH_201907.csv", encoding='cp949')
df_card2 = pd.read_csv("./data/CARD_SUBWAY_MONTH_202007.csv", encoding='cp949')
df_card3 = pd.read_csv("./data/CARD_SUBWAY_MONTH_202107.csv", encoding='cp949')

In [ ]:
df_card = pd.concat([df_card1,df_card2,df_card3]).reset_index(drop=True)

In [ ]:
pd.read_csv("../../CARD_SUBWAY_MONTH_202305.csv")

#### [참고] glob과 for 반복문을 활용한 복수 데이터 처리

**glob** 라이브러리의 *glob()* 을 활용하면 복수의 데이터 경로를 손쉽게 처리 가능

In [ ]:
# 대상 파일 목록 생성
from glob import glob

file_list  = glob('data/apt/*.csv')
file_list

In [ ]:
# for를 활용한 반복
a = list()
for path_ in file_list:
    a.append(pd.read_csv(path_, skiprows=15, encoding='CP949'))
a

In [ ]:
# 최종 작업
df_subway = pd.concat(a).reset_index(drop=True)
df_subway

In [ ]:
a = [pd.read_csv(path_, skiprows=15, encoding='CP949') for path_ in glob('data/apt/*.csv')]
df_subway = pd.concat(a).reset_index(drop=True)

<br>

### 2.2. merge()를 활용한 KEY 변수 기준 결합 

SQL의 JOIN, Excel의 VLOOKUP()과 같이 KEY 변수를 활용한 데이터 결합은 *merge()* 를 활용

In [ ]:
# 예제 데이터 불러오기
df_left  = pd.read_csv('data/data_left.csv')
df_right = pd.read_csv('data/data_right.csv')

In [ ]:
df_left

In [ ]:
df_right

<br>

> key를 활용한 데이터 결합에서는 일치하는 key가 있는, 짝이 있는 관측치만 출력하는 것이 기본값으로 설정되어 있습니다. SQL에서는 이것을 **inner join**이라고 부릅니다.  

*merge()* 에서 `how=` 옵션을 활용해서 다음과 같은 데이터 결합 방법 지정 

+ `inner`: inner join. key 기준 일치하는 관측치만 포함
+ `left`:  left join. inner join의 결과물과 왼쪽 데이터의 짝 없는 관측치 포함
+ `right`: right join. inner join의 결과물과 오른쪽 데이터의 짝 없는 관측치 포함
+ `outer`: full outer join. inner join과 양쪽 데이터의 짝이 없는 모든 관측치 포함

In [ ]:
# merge()를 활용한 결합
pd.merge(df_left, df_right, how='inner', on='category')

In [ ]:
# left join
pd.merge(df_left, df_right, how='left', on='category')

In [ ]:
# right join
pd.merge(df_left, df_right, how='right', on='category')

In [ ]:
# full outer join
pd.merge(df_left, df_right, how='outer', on='category', sort=True)

<br>
<hr>
<br>


## 3. 데이터 부분 선택

일반적인 비즈니스 데이터 분석에서 주제와 기간, 사이트, 제품, 공정 등 본인의 업무와 관련이 있는 일부 데이터만 선택하고 활용  
SQL을 활용한 데이터 추출 과정과 별개로 Python에서 각 분석 과정에서 맞게 부분 데이터를 다시 선택하고 사용

<br> 

In [ ]:
# 예제 데이터 불러오기
import pandas as pd
df_ins = pd.read_csv('data/insurance.csv')
df_ins.head()

<br>

### 3.1. .을 활용한 변수 선택

DataFrame 뒤에 마침표(.)를 찍고 `Tab` 키를 눌러 DataFrame의 메서드들과 함께 변수이름을 확인 가능  
.은 가장 간단한 변수 선택 방법이며 선택된 변수는 **Series** 형식으로 출력  

In [ ]:
# .을 활용한 하나의 변수 선택
df_ins.bmi

<br>


### 3.2. 대괄호를 활용한 데이터 부분 선택

DataFrame에 대괄호를 붙이고 슬라이스:로 관측치 번호를 지정하거나 따옴표''로 변수 이름을 넣어 데이터 부분을 선택 가능  
변수 이름을 리스트 형식으로 묶어 넣어 여러개 변수를 한번에 선택 가능

In [ ]:
# 관측치 선택
df_ins.iloc[0:3]

In [ ]:
df_ins.loc[0:2]

In [ ]:
# 한 변수 선택 
df_ins['age']

In [ ]:
# 리스트를 활용한 복수 변수 선택
df_ins[ ['age','smoker','charges'] ]

In [ ]:
# 연속된 대괄호 활용가능
df_ins[0:5][['age','smoker','charges']]

<br>

#### [실습]  

1. 아래의 명령어를 실행해서 df_subway 데이터 생성하기 

2. .columns 메서드를 활용해서 변수이름 확인하기

3. 슬라이스를 활용하여 11~15번째 관측치 선택하기

4. '사용일자', '역명', '하차총승객수' 세 변수 선택하기



    

In [ ]:
df_subway = pd.read_csv('./data/CARD_SUBWAY_MONTH_202107.csv', encoding='CP949')
df_subway

In [ ]:
df_subway.columns

In [ ]:
df_subway[10:15]

In [ ]:
df_subway[['사용일자', '역명', '하차총승객수']]

<br>

## 3.3. loc과 iloc을 활용한 관측치/변수 선택

loc은 행 이름(index)과 열 이름(column)으로 데이터에서 일부를 선택하고, iloc은 정수(integer) 형식의 행 번호, 열 번호를 활용  
두 방법 모두 리스트[ ]나 슬라이스:를 활용한 방법을 지원



In [ ]:
# 실습을 위해 원본 데이터를 복제(copy)하고 부분선택
df_ins2 = df_ins.copy()[0:10]
df_ins2

In [ ]:
# 실습을 위해 인덱스를 별도로 지정
df_ins2['idx'] = list(range(111, 131, 2))

df_ins2.set_index('idx', inplace=True)
df_ins2 = df_ins2.set_index('idx')

df_ins2

In [ ]:
# 실습을 위해 원본 데이터를 복제(copy)하고 부분선택
df_ins3 = df_ins[0:10]
df_ins3

In [ ]:
df_ins3.loc[0, 'age'] = 20
df_ins

<br> 

### 3.3.1. loc을 활용한 부분 선택

loc은 실제로 눈에 보이는 index와 column을 활용

In [ ]:
df_ins2

In [ ]:
df_ins2[0:3]

In [ ]:
df_ins2.loc[0:2]

In [ ]:
df_ins2.loc[111]

In [ ]:
df_ins2.loc[[111]]

In [ ]:
df_ins2.loc[101:103]

In [ ]:
df_ins2.loc[101:103, 'smoker']

In [ ]:
# 변수이름 리스트 활용가능
df_ins2.loc[101:103, ['smoker','region']]

In [ ]:
# 변수이름 슬라이스:를 활용 가능 
df_ins2.loc[101:103, 'smoker':'charges']

In [ ]:
# 모든 관측치 선택할 때는 :
df_ins2.loc[:, 'smoker':'charges']

<br> 

### 3.2.2. iloc을 활용한 부분 선택

iloc은 이름과 상관없이 정수로 표현한 위치, 번호를 활용하며 리스트나 슬라이스 활용 방법은 loc과 동일

In [ ]:
df_ins2.iloc[0:3, [0,3,4]]

In [ ]:
df_ins2.iloc[0:3]

In [ ]:
df_ins2[0:3]

#### [실습] 

1. df_pr에서 index 기준 '3'의 'Weight' 확인하기
2. df_pr에서 index 기준 '11~15'의 'Age'부터 'Exercise'까지 선택하기
3. df_pr에서 첫번째 ~ 다섯번째 관측치와 다섯번째 ~ 열번째 변수 선택하기

In [ ]:
df_pr = pd.read_csv('data/PulseRates.csv')
df_pr.head()

In [ ]:
df_pr.loc[3, 'Weight']

In [ ]:
df_pr.loc[11:15, 'Age':'Exercise']

In [ ]:
df_pr.iloc[0:5, 4:10]

### 3.4. 함수를 활용한 여러 변수 선택 



In [ ]:
df_ins.head(1)

In [ ]:
df_ins['ager'] = df_ins['age']
df_ins[['age']]

In [ ]:
# filter( ) 메서드에서 변수 이름 패턴을 활용한 선택 
df_ins.filter(regex='^age$')
    ## regex :  정규표현식(regular expression)
    ## '^s' : 's'로 시작하는 이름/텍스트,  ^시작  $끝
    

In [ ]:
# 변수형식 확인하기
df_ins.dtypes
    ## int/float : 숫자
    ## object : 문자열

In [ ]:
# 수치형 변수만 선택
df_ins.select_dtypes(include='number')

In [ ]:
# 문자열 변수만 선택
df_ins.select_dtypes(include='object')

<br>

#### [실습] Student performance 데이터 활용

1. df_sp에서 수치형 변수만 선택
2. df_sp에서 문자열 변수만 선택
3. df_sp에서 이름에 'score'가 들어간 변수만 선택


In [ ]:
df_sp = pd.read_csv('data/StudentsPerformance.csv')
df_sp.head()

In [ ]:
df_sp.select_dtypes(include='number')

In [ ]:
df_sp.select_dtypes(include='object')

In [ ]:
df_sp.filter(regex='score')

<br>

### 3.5. 조건을 활용한 관측치 선택

SQL에서 WHERE 절이나 Excel의 Filter와 같이 데이터에서 부분을 선택할 때 조건을 활용하는 경우 많음  
[ ]나 .loc[ ] 안에 조건식을 넣어서 조건과 일치하는 관측치만 선택 가능

In [ ]:
# 1 단계 : 조건 설정(결과는 True/False)
    # bool 타입 Series 
df_ins['age'] < 30

In [ ]:
# 2 단계 : []와 조건을 활용한 관측치 선택
df_ins[ df_ins['age'] < 30 ]

In [ ]:
# &와 |를 활용한 조건 결합
df_ins[(df_ins['age'] < 30) & (df_ins['sex'] == 'female')]

In [ ]:
df_ins[(df_ins['age'] < 30) | (df_ins['sex'] == 'female')]

<br> 

> 특히 비즈니스 데이터는 범주화, 그룹화된 변수들이 많고, 수많은 담당자들이 그 중 일부 범주, 그룹, 수준을 나눠서 운영하는 경우가 많습니다.  
*isin()* 을 활용해서 내가 관심있는 범주인지 아닌지 포함여부에 대한 연산이 가능합니다.

In [ ]:
# 변수 region의 수준 목록 확인 및 관심 수준 선택
df_ins['region'].unique()

In [ ]:
# isin()을 활용한 특정 수준 관측치 선택
cond1 = df_ins['region'].isin(['southeast','northwest'])
cond1

In [ ]:
df_ins[cond1]

In [ ]:
df_ins[df_ins['region'].isin(['southeast','northwest'])]

In [ ]:
df_ins['reg_tf'] = cond1
df_ins

<br>

#### [실습]

1. df_sp에서 math score가 90 이상인 관측치 선택
2. df_sp에서 race/ethnicity가 'group D', 'group E'인 관측치 선택(isin() 활용)
3. 1.과 2.를 동시에 만족하는 관측치 선택 

In [ ]:
df_sp.head()

In [ ]:
cond_math = df_sp['math score'] >= 90
df_sp[cond_math]

In [ ]:
cond_race = df_sp['race/ethnicity'].isin(['group D', 'group E'])
df_sp[cond_race]

In [ ]:
df_sp[cond_math & cond_race]

#### [참고] Series의 str 메서드 활용
문자열 Series(한 변수)에서 str 함수를 활용하면 특정 단어를 포함하거나 특정 패턴과 일치하는 관측치를 선택 가능

In [ ]:
df_sp

In [ ]:
df_sp['parental level of education'].str.startswith('b')

In [ ]:
df_sp['parental level of education'].str.endswith('college')

In [ ]:
df_sp['parental level of education'].str.contains('degree')

<br>

#### [참고] Series의 between 메서드 활용
수치형 Series(한 변수)에서 *between()* 으로 특정 범위 내 관측치 선택 가능

In [ ]:
df_sp['math score'].between(80, 89.9)

In [ ]:
# 양쪽 끝 경계 포함 여부 지정 가능
    # 'both', 'left', 'right'
df_sp[df_sp['math score'].between(80, 90, inclusive='left')] 

<br>

#### [참고] ~를 활용한 부정(True/False 반전)
bool Series(True/False) 앞에 **~** 를 붙여서 True와 False를 뒤집기 가능

In [ ]:
cond1 = df_sp['math score'].between(80, 90, inclusive='left')
cond1

In [ ]:
~cond1

In [ ]:
df_sp[~cond1]

<br>

### 3.6. 함수를 활용한 부분 관측치 선택


In [ ]:
# head( )와 tail()
df_ins.head()
df_ins.tail()

In [ ]:
# sample( )의 활용
#df_ins.sample(frac=0.01)
df_ins.sample(n=10)

In [ ]:
# nlargest( ), nsmallest( )로 상위/하위 관측치 선택
df_ins.nlargest(50, 'charges')


In [ ]:
df_ins.nsmallest(10, 'charges')

<br>

#### [실습]

1. df_sp에서 math score 상위 20 명 선택
2. df_sp에서 writing score 하위 10명 선택


In [ ]:
df_sp.sample(n=10)

In [ ]:
df_sp.nlargest(20, 'math score')

In [ ]:
df_sp.nsmallest(10, 'writing score')

<br>

### 3.7. 중복값 제거

`drop_duplicates()`를 활용해서 중복값을 제거한 목록 생성 가능

In [126]:
df_ins[['sex','region']].drop_duplicates()

,sex,region
0,female,southwest
1,male,southeast
3,male,northwest
5,female,southeast
7,female,northwest
8,male,northeast
12,male,southwest
16,female,northeast


### 3.8. 관측치 정렬

`sort_values()`를 활용해서 관측치를 정렬

In [127]:
# age 순 데이터 정렬
df_ins.sort_values('age')

,age,sex,bmi,children,smoker,region,charges,ager,reg_tf
1248,18,female,39.820,0,no,southeast,1633.96180,18,True
529,18,male,25.460,0,no,northeast,1708.00140,18,False
564,18,female,32.120,2,no,southeast,2801.25880,18,True
571,18,female,37.290,1,no,southeast,2219.44510,18,True
600,18,female,39.160,0,no,southeast,1633.04440,18,True
...,...,...,...,...,...,...,...,...,...
398,64,male,25.600,2,no,southwest,14988.43200,64,False
335,64,male,34.500,0,no,southwest,13822.80300,64,False
378,64,female,30.115,3,no,northwest,16455.70785,64,True
1265,64,male,23.760,0,yes,southeast,26926.51440,64,True


In [128]:
# 원본 데이터는 영향 없음
df_ins.head()

,age,sex,bmi,children,smoker,region,charges,ager,reg_tf
0,20,female,27.900,0,yes,southwest,16884.92400,20,False
1,18,male,33.770,1,no,southeast,1725.55230,18,True
2,28,male,33.000,3,no,southeast,4449.46200,28,True
3,33,male,22.705,0,no,northwest,21984.47061,33,True
4,32,male,28.880,0,no,northwest,3866.85520,32,True


In [131]:
# 원본 데이터의 정렬
df_ins = df_ins.sort_values('age')
#df_ins.sort_values('age', inplace=True)
df_ins.head()

,age,sex,bmi,children,smoker,region,charges,ager,reg_tf
1248,18,female,39.820,0,no,southeast,1633.961800,18,True
121,18,male,23.750,0,no,northeast,1705.624500,18,False
399,18,female,38.170,0,no,southeast,1631.668300,18,True
427,18,female,29.165,0,no,northeast,7323.734819,18,False
442,18,male,43.010,0,no,southeast,1149.395900,18,True


In [132]:
# 내림차순 지정
df_ins = df_ins.sort_values('age', ascending=False)
df_ins.head()

,age,sex,bmi,children,smoker,region,charges,ager,reg_tf
635,64,male,38.19,0,no,northeast,14410.9321,64,False
534,64,male,40.48,0,no,southeast,13831.1152,64,True
199,64,female,39.33,0,no,northeast,14901.5167,64,False
420,64,male,33.88,0,yes,southeast,46889.2612,64,True
418,64,male,39.16,1,no,southeast,14418.2804,64,True


In [133]:
# 복수 기준의 설정 
df_ins.sort_values(['age', 'charges'], ascending=[True, False])

,age,sex,bmi,children,smoker,region,charges,ager,reg_tf
803,18,female,42.240,0,yes,southeast,38792.68560,18,True
759,18,male,38.170,0,yes,southeast,36307.79830,18,True
161,18,female,36.850,0,yes,southeast,36149.48350,18,True
623,18,male,33.535,0,yes,northeast,34617.84065,18,False
57,18,male,31.680,2,yes,southeast,34303.16720,18,True
...,...,...,...,...,...,...,...,...,...
768,64,female,39.700,0,no,southwest,14319.03100,64,False
801,64,female,35.970,0,no,southeast,14313.84630,64,True
752,64,male,37.905,0,no,northwest,14210.53595,64,True
534,64,male,40.480,0,no,southeast,13831.11520,64,True


In [135]:
# index를 활용한 정렬
df_ins = df_ins.sort_index(ascending=False)
df_ins.head()

,age,sex,bmi,children,smoker,region,charges,ager,reg_tf
1337,61,female,29.07,0,yes,northwest,29141.3603,61,True
1336,21,female,25.80,0,no,southwest,2007.9450,21,False
1335,18,female,36.85,0,no,southeast,1629.8335,18,True
1334,18,female,31.92,0,no,northeast,2205.9808,18,False
1333,50,male,30.97,3,no,northwest,10600.5483,50,True


<br>


#### [실습] 데이터 df_sp 활용

1. 전체 관측치를 'math score', 'reading score'의 내림차순으로 정렬해서 출력


In [137]:
df_sp.sort_values(['math score', 'reading score'], ascending=[False, False]).head(10)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
149,male,group E,associate's degree,free/reduced,completed,100,100,93
458,female,group E,bachelor's degree,standard,none,100,100,100
916,male,group E,bachelor's degree,standard,completed,100,100,100
962,female,group E,associate's degree,standard,none,100,100,100
625,male,group D,some college,standard,completed,100,97,99
623,male,group A,some college,standard,completed,100,96,86
451,female,group E,some college,standard,none,100,92,97
114,female,group E,bachelor's degree,standard,completed,99,100,100
263,female,group E,high school,standard,none,99,93,90
306,male,group E,some college,standard,completed,99,87,81


#### End of script